In [55]:
import os
import cv2
import numpy as np
from imutils import face_utils
import dlib

# Image Preprocessing 

In [ ]:
input_folder = "archive/train"
processed_folder = "D:\\2. College project\\drowsiness detection while driving-sem5\\driver_drowsiness_system_CNN\\processed_images"
face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml') 
predictor = dlib.shape_predictor('data/shape_predictor_68_face_landmarks.dat')

In [ ]:
def preprocess_yawn_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    detector = dlib.get_frontal_face_detector()
    faces = detector(gray, 1)
    if len(faces) == 0:
        print(f"No face detected in image: {image_path}")
        return None
    
    for face in faces:
        rect = dlib.rectangle(face.left(), face.top(), face.right(), face.bottom())
        landmarks = predictor(gray, rect)
        landmarks = face_utils.shape_to_np(landmarks)

        mouth_points = landmarks[48:68]
        
        (x_min, y_min) = mouth_points.min(axis=0)
        (x_max, y_max) = mouth_points.max(axis=0)
        
        # Crop area mulut
        mouth = gray[y_min:y_max, x_min:x_max]
        
        mouth_resized = cv2.resize(mouth, (64, 64))
        mouth_normalized = mouth_resized / 255.0
        mouth_final = np.expand_dims(mouth_normalized, axis=-1)
        
        return mouth_final

In [ ]:
def preprocess_eye_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return None

    eye_resized = cv2.resize(img, (64, 64))
    eye_normalized = eye_resized / 255.0
    eye_final = np.expand_dims(eye_normalized, axis=-1)
    
    return eye_final

In [ ]:
def process_and_save_images(input_folder, processed_folder):
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    for category in os.listdir(input_folder):
        category_path = os.path.join(input_folder, category)
        output_category_folder = os.path.join(processed_folder, category)
        if not os.path.exists(output_category_folder):
            os.makedirs(output_category_folder)
        
        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)

            if category in ['yawn', 'no_yawn']:
                processed_image = preprocess_yawn_image(image_path)
            elif category in ['Closed', 'Open']:
                processed_image = preprocess_eye_image(image_path)
            else:
                print(f"Skipped unknown category: {category}")
                continue
            
            if processed_image is not None:
                processed_image = (processed_image * 255).astype(np.uint8)
                output_path = os.path.join(output_category_folder, filename)
                cv2.imwrite(output_path, processed_image)
                print(f"Processed and saved: {output_path}")

process_and_save_images(input_folder, processed_folder)

Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed\_0.jpg
Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed\_1.jpg
Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed\_10.jpg
Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed\_100.jpg
Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed\_101.jpg
Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed\_102.jpg
Processed and saved: D:\2. College project\drowsiness detection while driving-sem5\driver_drowsiness_system_CNN\processed_images\Closed

# Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,          
    rotation_range=10,            
    width_shift_range=0.1,       
    height_shift_range=0.1,    
    shear_range=0.1,              
    zoom_range=0.1,              
    horizontal_flip=True,      
    brightness_range=[0.9, 1.1],  
    validation_split=0.2         
)

train_generator = train_datagen.flow_from_directory(
    processed_folder,
    target_size=(64, 64),   
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',   
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    processed_folder,
    target_size=(64, 64),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 2222 images belonging to 4 classes.
Found 554 images belonging to 4 classes.


# Architecture CNN

In [61]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Evaluate accuracy model